In [9]:
import requests
import bs4
import pandas as pd
from io import StringIO

## UFC Fighter Dataset

In [4]:
#scrape from ufc stats
webpage_source = requests.get("http://ufcstats.com/statistics/fighters?char=a&page=all").text

In [36]:
data = pd.read_html(StringIO(webpage_source))
fighter_df = data[0] 

#remove empty row
fighter_df.drop(labels=0, axis=0, inplace=True) 

#rename columns
colnames = ['First', 'Last', 'Nickname', 'Height', 'Weight', 'Reach', 'Stance', 'Win', 'Loss', 'Draw', 'Belt']
fighter_df.columns = colnames


fighter_df

,First,Last,Nickname,Height,Weight,Reach,Stance,Win,Loss,Draw,Belt
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28.0,4.0,0.0,NaN
4,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN
5,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
216,Abu,Azaitar,Captain Morocco,"5' 9""",185 lbs.,"76.0""",Orthodox,14.0,4.0,1.0,NaN
217,Ottman,Azaitar,Bulldozer,"5' 8""",155 lbs.,"71.0""",Switch,13.0,2.0,0.0,NaN
218,Luiz,Azeredo,NaN,"5' 9""",154 lbs.,--,Orthodox,15.0,10.0,0.0,NaN
219,Luciano,Azevedo,NaN,"6' 3""",161 lbs.,--,Orthodox,17.0,9.0,1.0,NaN
